In [2]:
import sys 
from pathlib import Path
import numpy as np

cwd = Path.cwd()
if  (cwd / 'src').exists():              
    src_dir = cwd / 'src'
elif(cwd.parent / 'src').exists():        
    src_dir = cwd.parent / 'src'
else:
    raise FileNotFoundError("No pude encontrar la carpeta 'src/'")

sys.path.insert(0, str(src_dir))

from layers.conv2d      import Conv2D
from layers.activations import ReLU
from layers.pooling     import MaxPool2D
from layers.dense       import *

from train import *
from model import *

#### You can create a model manually like this and then perfome some forwardpass: 

In [8]:

model = [
  Conv2D(n_C_prev=3, n_C=8,  f=3, stride=1, pad=1),
  ReLU(),
  MaxPool2D(f=2, stride=2),

  Conv2D(n_C_prev=8, n_C=16, f=3, stride=1, pad=1),
  ReLU(),
  MaxPool2D(f=2, stride=2),
  
  Flatten(),
  Dense(n_units=12, initialization='he')] # 12 clases 

X_batch = np.random.randn(2, 8, 8, 3).astype(np.float32)
batch_size  = X_batch.shape[0]
num_classes = 12

y_int = np.random.randint(0, num_classes, size=batch_size)
y_batch = np.zeros((batch_size, num_classes), dtype=np.float32)
y_batch[np.arange(batch_size), y_int] = 1.0

print("y_batch shape:", y_batch.shape)
print("Sample label indices:", y_int[:5])
print("Sample one‑hot rows:\n", y_batch[:5])

A_out, all_caches = conv_net_forward(model, X_batch)

print("Output shape:", A_out.shape)
print("Keys in caches:", list(all_caches.keys()))


y_batch shape: (2, 12)
Sample label indices: [3 4]
Sample one‑hot rows:
 [[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
Output shape: (2, 12)
Keys in caches: ['A_prev1 conv2d', 'W1 conv2d', 'b1 conv2d', 'hparameters1 conv2d', 'Z1 ReLu', 'A_prev1 PoolLayer', 'hparameters1 PoolLayer', 'A_prev2 conv2d', 'W2 conv2d', 'b2 conv2d', 'hparameters2 conv2d', 'Z2 ReLu', 'A_prev2 PoolLayer', 'hparameters2 PoolLayer', 'A_prev1 dense', 'A1 dense']


### Try cost calculation 

In [9]:
compute_cost(A_out , y_batch , 'CrossEntropy')

1.1134008620680533

### You can see all the post and pre activations parameters, biases and hyperparameters storaged in the all_caches forward pass 

In [7]:
all_caches['A_prev1 conv2d']

array([[[[ 3.66120994e-01, -1.49204183e+00, -2.68377990e-01],
         [ 7.30599701e-01, -2.21511126e-01, -5.57281673e-01],
         [-1.09825480e+00,  7.51788318e-01,  8.08050752e-01],
         [-1.02063763e+00, -1.61946857e+00, -2.93554842e-01],
         [ 5.72569489e-01,  9.80663478e-01,  1.56596541e-01],
         [ 4.29040104e-01, -8.35425854e-01, -3.95397663e-01],
         [ 5.21071516e-02, -8.00177813e-01, -8.47889006e-01],
         [ 8.73454392e-01, -1.43239117e+00,  2.70992517e-01]],

        [[-3.27093005e-02, -1.06771028e+00, -6.63310826e-01],
         [ 2.69875050e-01,  1.38821810e-01, -2.64762819e-01],
         [-2.43377134e-01, -2.08106542e+00,  2.16399980e+00],
         [-3.07297230e-01, -6.08341753e-01, -8.21876884e-01],
         [ 7.59610310e-02, -1.08946049e+00, -1.49100256e+00],
         [-1.03441036e+00,  2.22136307e+00, -8.13085198e-01],
         [ 1.27823842e+00,  5.72304249e-01,  1.35106254e+00],
         [ 3.56341749e-01, -1.17065871e+00,  1.35646164e+00]],

    

### Try backprop

In [12]:
grads = conv_net_backward(model , A_out , y_batch , 0.001)
print("Keys in grads:", list(grads.keys()))

Keys in grads: ['dW1 dense', 'db1 dense', 'dA2 flatten', 'dA1 PoolLayer', 'dZ1 ReLu', 'dW1 conv2d', 'db1 conv2d', 'dA2 PoolLayer', 'dZ2 ReLu', 'dW2 conv2d', 'db2 conv2d']


In [14]:
grads['dW2 conv2d']

array([[[[ 4.30104142e-02, -3.64727025e-02,  1.18345262e-01,
          -1.27176419e-02,  3.84765160e-02, -1.32041371e-02,
           2.85984839e-02,  1.90060661e-02],
         [ 1.12508287e-02, -2.53881597e-03, -3.14764301e-03,
          -2.64239186e-02, -1.37898389e-01,  5.07502506e-02,
           6.58627866e-03, -3.65934753e-02],
         [-7.88626414e-02, -4.60856363e-02, -2.55800642e-02,
           3.11285163e-02, -3.98378403e-02, -4.33551193e-03,
          -1.83733114e-02,  7.53060536e-03]],

        [[-6.71222007e-02, -1.08844681e-04, -1.17958693e-01,
          -1.25192365e-02,  5.82930622e-02, -4.73590271e-02,
          -7.93311448e-02,  1.92821929e-02],
         [-2.97179797e-02,  4.22737375e-02, -6.99750312e-02,
          -3.45639040e-02, -5.90994411e-02, -4.49692342e-02,
          -4.20476406e-02, -5.97922151e-02],
         [-1.32931636e-02, -5.81117688e-02, -1.11792566e-01,
           1.42789609e-02, -5.79041747e-02, -6.71059557e-02,
          -3.17435771e-02, -6.90093887e-0

## Use the built function to create the model:

In [16]:
filters = [X_batch.shape[3] , 8 , 16] 
pool = [1 , 1 ,1]
batch_size = 16
lr = 0.001

model = crear_modelo(filters=filters, pool=pool, n_classes=2)
print(*model, sep='\n')